# 如何将ZAIR的数据转换为Stim格式的数据。

In [1]:
import logging
import json

# 配置日志
logging.basicConfig(level=logging.WARNING, format='%(levelname)s: %(message)s')

class ZAIR2STIM:
    """
    ZAIR 指令集到带有噪声模型的 Stim 线路的转换器。
    时间单位均为微秒 (us)。
    """
    
    def __init__(self, 
                 p_1q: float = 3e-4, 
                 p_2cz: float = 5e-3, 
                 p_2idea: float = 2.5e-3, 
                 p_tran: float = 1e-3, 
                 t2_time: float = 1.5e6, # T2时间 (us)
                 t_tran: float = 15.0, # 单次原子转换时间 (us)
                 t_ryd: float = 0.36, # Rydberg 门操作时间 (us)
                 t_1q: float = 52.0, # 单量子比特门操作时间 (us)
                 entanglement_zone: set = {1, 2}, # 纠缠区 SLM 阵列 ID
                 include_comments: bool = True # 新增参数
                ):
        """
        初始化转换器参数。
        """
        self.p_1q = p_1q
        self.p_2cz = p_2cz
        self.p_2idea = p_2idea
        self.p_tran = p_tran
        self.t2_time = t2_time
        self.t_tran = t_tran
        self.t_ryd = t_ryd
        self.t_1q = t_1q
        self.entanglement_zone = entanglement_zone
        self.include_comments = include_comments

        # 编译状态
        self.circuit_name: str = ""
        self.architecture_spec_path: str = ""
        self.stim_circ: str = ""
        self.num_qubits: int = 0
        self.set_all_qubit: set = set()
        
    def _add_stim_line(self, line: str):
            """向 Stim 线路中添加一行指令或注释，受 self.include_comments 控制。"""
            stripped_line = line.strip()
            is_comment = stripped_line.startswith("#")

            if is_comment and not self.include_comments:
                return 
            
            self.stim_circ += stripped_line + "\n"

    def _get_qubits_str(self, qubits: set | list) -> str:
        """将比特集合或列表转换为 Stim 格式的字符串。"""
        if not qubits:
            return ""
        # 确保是排序后的比特，以保证输出的一致性
        return " ".join(map(str, sorted(list(qubits))))

    def init2stim(self):
        """
        添加 Reset 操作和 TICK。
        """
        logging.info("--> Processing INIT: Reset all qubits.")
        qubit_str = self._get_qubits_str(self.set_all_qubit)
        self._add_stim_line(f"R {qubit_str}")
        self._add_stim_line("TICK")
        
    def rearrangeJob2stim(self, logical_instruction: dict, set_qubit_in_rydberg: set):
        """
        添加原子转换噪声和空闲比特退相干噪声，并更新 Rydberg 区集合。
        """
        rearrange_during_time = logical_instruction["end_time"] - logical_instruction["begin_time"]
        aod_qubits = set(logical_instruction["aod_qubits"])
        qubits_idle = self.set_all_qubit - aod_qubits
        
        logging.info(f"--> Processing REARRANGEJOB {logical_instruction['id']}: Time={rearrange_during_time:.2f} us. Qubits: {len(aod_qubits)}")

        # 1. AOD 转换噪声 (双次：SLM->AOD 和 AOD->SLM)
        qubit_str_aod = self._get_qubits_str(aod_qubits)
        if qubit_str_aod:
            self._add_stim_line(f"# SLM to AOD (p={self.p_tran:.2e})")
            self._add_stim_line(f"DEPOLARIZE1({self.p_tran}) {qubit_str_aod}")
            self._add_stim_line(f"# AOD to SLM (p={self.p_tran:.2e})")
            self._add_stim_line(f"DEPOLARIZE1({self.p_tran}) {qubit_str_aod}")
        
        # 2. 其他空闲比特退相干噪声
        if qubits_idle:
            p_idle = rearrange_during_time / self.t2_time
            qubit_str_idle = self._get_qubits_str(qubits_idle)
            logging.info(f"    - Idle Coherence p={p_idle:.2e} for {len(qubits_idle)} qubits.")
            self._add_stim_line(f"# Idle Coherence (p={p_idle:.2e})")
            self._add_stim_line(f"DEPOLARIZE1({p_idle}) {qubit_str_idle}")

        self._add_stim_line("TICK")
        
        # 3. 更新 Rydberg 区集合
        list_qubit_end_location = logical_instruction["end_locs"]
        for q_idx, new_array, _, _ in list_qubit_end_location:
            if q_idx in aod_qubits: # 只有被移动的比特才需要更新位置信息
                if new_array in self.entanglement_zone:
                    set_qubit_in_rydberg.add(q_idx)
                else:
                    set_qubit_in_rydberg.discard(q_idx)
        logging.info(f"    - Updated set_qubit_in_rydberg: {sorted(list(set_qubit_in_rydberg))}")

    def rydberg2stim(self, logical_instruction: dict, set_qubit_in_rydberg: set):
        """
        添加 CZ 门操作和相应的噪声。
        """
        gates = logical_instruction["gates"]
        cz_pairs = []
        qubits_cz = set() # 作用双比特门的比特集合
        
        for gate in gates:
            q0, q1 = gate["q0"], gate["q1"]
            cz_pairs.append(f"{q0} {q1}")
            qubits_cz.add(q0)
            qubits_cz.add(q1)

        logging.info(f"--> Processing RYDBERG {logical_instruction['id']}: CZ count={len(cz_pairs)}")

        # 1. 门操作
        self._add_stim_line(f"CZ {' '.join(cz_pairs)}")
        
        # 2. 噪声计算
        qubits_idle_ryd = set_qubit_in_rydberg - qubits_cz # 纠缠区空闲比特
        qubits_idle_other = self.set_all_qubit - set_qubit_in_rydberg # 其他区空闲比特
        
        # 2a. CZ 门操作噪声
        if cz_pairs:
            cz_qubits_str = " ".join(cz_pairs)
            self._add_stim_line(f"# CZ Gate Noise (p={self.p_2cz:.2e})")
            self._add_stim_line(f"DEPOLARIZE2({self.p_2cz}) {cz_qubits_str}")

        # 2b. Rydberg 区空闲比特噪声
        if qubits_idle_ryd:
            ryd_idle_str = self._get_qubits_str(qubits_idle_ryd)
            self._add_stim_line(f"# Rydberg Zone Idle Noise (p={self.p_2idea:.2e})")
            self._add_stim_line(f"DEPOLARIZE1({self.p_2idea}) {ryd_idle_str}")
            
        # 2c. 其他空闲比特退相干噪声
        if qubits_idle_other:
            p_idle_other = self.t_ryd / self.t2_time
            other_idle_str = self._get_qubits_str(qubits_idle_other)
            self._add_stim_line(f"# Other Idle Coherence (p={p_idle_other:.2e})")
            self._add_stim_line(f"DEPOLARIZE1({p_idle_other}) {other_idle_str}")

        self._add_stim_line("TICK")

    def singleqGate2stim(self, logical_instruction: dict):
        """
        添加单比特门操作和相应的噪声。
        """
        gates = logical_instruction["gates"]
        single_qubit_during_time = logical_instruction["end_time"] - logical_instruction["begin_time"]
        
        qubits_op = set()
        gates_list = []
        gate_name = ""
        
        for gate in gates:
            q = gate["q"]
            gate_name = gate["name"].upper() # 假设所有门类型相同
            qubits_op.add(q)
            gates_list.append(str(q))
        
        qubits_idle = self.set_all_qubit - qubits_op
        qubit_str_op = " ".join(gates_list)
        qubit_str_idle = self._get_qubits_str(qubits_idle)
        
        logging.info(f"--> Processing 1QGATE {logical_instruction['id']}: {gate_name} count={len(qubits_op)}, Time={single_qubit_during_time:.2f} us.")

        # 1. 门操作
        if qubit_str_op:
            self._add_stim_line(f"{gate_name} {qubit_str_op}")

        # 2. 噪声计算
        # 2a. 操作比特的门操作噪声
        if qubit_str_op:
            self._add_stim_line(f"# 1Q Gate Noise (p={self.p_1q:.2e})")
            self._add_stim_line(f"DEPOLARIZE1({self.p_1q}) {qubit_str_op}")
            
            # 2b. 操作比特的空闲噪声（T_1q 之外的时间）
            p_idle_op = (single_qubit_during_time - self.t_1q) / self.t2_time
            if p_idle_op > 0:
                 self._add_stim_line(f"# 1Q Op Idle Coherence (p={p_idle_op:.2e})")
                 self._add_stim_line(f"DEPOLARIZE1({p_idle_op}) {qubit_str_op}")

        # 2c. 空闲比特的退相干噪声（整个操作时间）
        if qubit_str_idle:
            p_idle_idle = single_qubit_during_time / self.t2_time
            self._add_stim_line(f"# Other Idle Coherence (p={p_idle_idle:.2e})")
            self._add_stim_line(f"DEPOLARIZE1({p_idle_idle}) {qubit_str_idle}")

        self._add_stim_line("TICK")


    def zair2stim(self, data: dict, num_qubits: int) -> str:
        """
        主函数：处理整个 ZAIR 数据结构并生成 Stim 线路。
        """
        # 初始化编译状态
        self.stim_circ = ""
        self.num_qubits = num_qubits
        self.set_all_qubit = set(range(num_qubits))
        self.circuit_name = data.get("name", "untitled_circuit")
        self.architecture_spec_path = data.get("architecture_spec_path", "unknown_arch")
        
        logging.info(f"Starting conversion for circuit: {self.circuit_name} with {num_qubits} qubits.")
        
        # 维护一个集合，记录当前位于纠缠区（Rydberg zone）的量子比特
        set_qubit_in_rydberg = set()
        
        # 获取指令列表
        list_instrcution = data.get("instructions", {})
        
        # --- 遍历指令 ---
        for logical_instruction in list_instrcution:
            inst_type = logical_instruction.get("type")
            
            try:
                if inst_type == "init":
                    self.init2stim()
                    # 初始化后，根据 init_locs 确定初始的 set_qubit_in_rydberg 集合
                    for q_idx, array_id, _, _ in logical_instruction.get("init_locs", []):
                        if array_id in self.entanglement_zone:
                            set_qubit_in_rydberg.add(q_idx)
                    logging.info(f"    - Initial set_qubit_in_rydberg: {sorted(list(set_qubit_in_rydberg))}")

                elif inst_type == "1qGate":
                    self.singleqGate2stim(logical_instruction)
                
                elif inst_type == "rearrangeJob":
                    self.rearrangeJob2stim(logical_instruction, set_qubit_in_rydberg)
                
                elif inst_type == "rydberg":
                    self.rydberg2stim(logical_instruction, set_qubit_in_rydberg)
                
                else:
                    logging.warning(f"Skipping unknown instruction type: {inst_type}")
                    self._add_stim_line(f"# Unknown instruction type: {inst_type}")
                    self._add_stim_line("TICK")
                    
            except Exception as e:
                logging.error(f"Error processing instruction {inst_type} (ID: {logical_instruction.get('id', 'N/A')}): {e}")
                # 遇到错误时，添加注释并跳过，以继续编译
                self._add_stim_line(f"# ERROR: Failed to process {inst_type} (ID: {logical_instruction.get('id', 'N/A')})")
                self._add_stim_line("TICK")

        logging.info(f"Conversion finished. Total lines in Stim circuit: {len(self.stim_circ.splitlines())}")
        return self.stim_circ

In [2]:
# ...existing code...
import os
import json
# import logging  # 移除logging导入
# from ZAIR2STIM import ZAIR2STIM  # 假设 ZAIR2STIM 类在 ZAIR2STIM.py 文件中，如果不在，请调整导入

# 配置日志 - 移除，使用print代替
# logging.basicConfig(level=logging.WARNING, format='%(levelname)s: %(message)s')

def parse_num_qubits_from_filename(filename: str) -> int:
    """
    从文件名中解析 num_qubits，例如 'hgp_code_n_441_k_9_r_1_cz_code.json' -> 441
    """
    # 假设文件名格式为 hgp_code_n_{n}_k_{k}_r_{r}_cz_code.json
    parts = filename.split('_')
    for i, part in enumerate(parts):
        if part == 'n' and i + 1 < len(parts):
            try:
                return int(parts[i + 1])
            except ValueError:
                pass
    raise ValueError(f"Cannot parse num_qubits from filename: {filename}")

def merge_stim_circuits(path_original: str, path_new: str) -> str:
    """
    从原始Stim文件（path_original）中提取测量噪声X_ERROR行以及后续的所有测量、detector和logical observable相关的行，
    并将其添加到新Stim电路文件（path_new）的末尾，返回合并后的Stim电路字符串。
    
    Args:
        path_original (str): 原始Stim文件的路径。
        path_new (str): 新Stim电路文件的路径。
    
    Returns:
        str: 合并后的Stim电路字符串。
    """
    # 读取原始Stim文件
    with open(path_original, 'r') as f:
        lines_original = f.readlines()
    
    # 读取新Stim电路文件
    with open(path_new, 'r') as f:
        lines_new = f.readlines()
    
    # 在原始文件中找到第一个MR或M（测量指令），然后向前查找TICK
    start_index = -1
    for i, line in enumerate(lines_original):
        stripped = line.strip()
        if stripped.startswith('MR ') or stripped.startswith('M '):
            # 找到MR或M，从这里向前查找TICK
            for j in range(i-1, -1, -1):
                if lines_original[j].strip() == 'TICK':
                    start_index = j+1
                    break
            else:
                start_index = i  # 如果没找到TICK，使用i
            break
    
    if start_index == -1:
        raise ValueError("No MR or M instruction found in the original file.")
    
    # 提取从start_index到末尾的所有行
    extracted_lines = lines_original[start_index:]

    # 删除以H开头的行以及其下一行噪声，因为前面已经作用过了
    filtered_lines = []
    i = 0
    while i < len(extracted_lines):
        stripped = extracted_lines[i].strip()
        if stripped.startswith('H '):
            # 跳过这一行和下一行（噪声）
            i += 2
        else:
            filtered_lines.append(extracted_lines[i])
            i += 1
    extracted_lines = filtered_lines    

    # 将MX和MZ替换为M
    for i in range(len(extracted_lines)):
        stripped = extracted_lines[i].strip()
        if stripped.startswith('MX '):
            extracted_lines[i] = stripped.replace('MX ', 'M ', 1) + '\n'
        elif stripped.startswith('MZ '):
            extracted_lines[i] = stripped.replace('MZ ', 'M ', 1) + '\n'
            
    # 将新电路的行和提取的行合并
    merged_lines = lines_new + extracted_lines
    
    # 返回合并后的字符串
    return ''.join(merged_lines)

def save_stim_circuit(stim_str: str, folder_path: str, filename: str = "circuit.stim"):
    """
    将Stim电路字符串转换为Stim对象，并保存为.stim文件到指定文件夹。
    
    Args:
        stim_str (str): Stim电路的字符串表示。
        folder_path (str): 保存文件的文件夹路径。
        filename (str): 文件名，默认为"circuit.stim"。
    """
    import stim
    # 确保文件夹存在
    os.makedirs(folder_path, exist_ok=True)
    
    # 从字符串创建Stim电路
    circuit = stim.Circuit(stim_str)
    
    # 构建完整路径
    full_path = os.path.join(folder_path, filename)
    
    # 保存到文件
    circuit.to_file(full_path)
    
    print(f"Stim circuit saved to {full_path}")

def parse_core_name(filename: str) -> str:
    """
    从文件名中解析核心名称，例如 'hgp_code_n_1225_k_25_r_1_X_cz_code.json' -> 'hgp_code_n_1225_k_25_r_1_X_cz'
    """
    if filename.endswith('.json'):
        base = filename[:-5]
        return base.rsplit('_code', 1)[0]
    else:
        raise ValueError(f"Invalid filename: {filename}")

def parse_num_qubits_from_core(core_name: str) -> int:
    """
    从核心名称中解析 num_qubits，例如 'hgp_code_n_1225_k_25_r_1_X_cz' -> 1225
    """
    parts = core_name.split('_')
    for i, part in enumerate(parts):
        if part == 'n' and i + 1 < len(parts):
            try:
                return int(parts[i + 1])
            except ValueError:
                pass
    raise ValueError(f"Cannot parse num_qubits from core_name: {core_name}")

def process_zair_files(json_dir: str, output_dir: str, original_stim_dir: str):
    """
    处理指定目录下的所有 ZAIR JSON 文件，转换为 Stim 格式，并保存。
    
    Args:
        json_dir (str): ZAIR JSON 文件目录。
        output_dir (str): 输出目录。
        original_stim_dir (str): 原始 Stim 文件目录，用于合并测量等。
    """
    # 确保输出目录存在
    os.makedirs(output_dir, exist_ok=True)
    
    # 遍历 JSON 文件
    for filename in os.listdir(json_dir):
        if filename.endswith('.json'):
            # 单独处理BB code：跳过合并
            if 'bb' not in filename:
                print(f"Skipping merge for Other code: {filename}")
                continue
            json_path = os.path.join(json_dir, filename)
            print(f"Processing {filename}")  # 取消注释如果需要
            
            # 解析核心名称
            try:
                core_name = parse_core_name(filename)
                num_qubits = parse_num_qubits_from_core(core_name)
            except ValueError as e:
                print(f"ERROR: Skipping {filename}: {e}")
                continue

            # 读取 JSON 数据
            with open(json_path, 'r') as f:
                data = json.load(f)
            
            # 解析 num_qubits
            try:
                num_qubits = parse_num_qubits_from_filename(filename)
            except ValueError as e:
                print(f"ERROR: Skipping {filename}: {e}")
                continue
            
            # 转换到 Stim
            converter = ZAIR2STIM(include_comments=False)
            stim_circuit = converter.zair2stim(data, num_qubits)
            
            # 保存 na_measure 版本
            na_without_measure_filename = f"{core_name}_na_without_measure.stim"
            na_without_measure_path = os.path.join(output_dir, na_without_measure_filename)
            print(f"Saving NA Stim circuit to {na_without_measure_path}")  # 取消注释如果需要
            with open(na_without_measure_path, 'w') as f:
                f.write(stim_circuit)
            
            # 查找对应的原始 Stim 文件
            original_stim_filename = f"{core_name.replace('_cz', '')}.stim"
            original_stim_path = os.path.join(original_stim_dir, original_stim_filename)
            print(f"Looking for original Stim file at {original_stim_path}")  # 取消注释如果需要

            if os.path.exists(original_stim_path):
                # 合并
                merged_stim_circuit = merge_stim_circuits(original_stim_path, na_without_measure_path)
                
                # 保存最终版本
                final_filename = f"{core_name}_na.stim"
                save_stim_circuit(merged_stim_circuit, output_dir, final_filename)
                print(f"Final Stim circuit saved as {final_filename}")  # 取消注释如果需要
            else:
                print(f"WARNING: Original Stim file not found: {original_stim_path}. Skipping merge.")

# 使用示例
json_dir = '/home/normaluser/ck/linkequantum-qLDPC/qLDPC/zac_result/trivial_placement_full_arch/code'
output_dir = '/home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_na_stim_circuit'
original_stim_dir = '/home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_stim_circuit'

process_zair_files(json_dir, output_dir, original_stim_dir)


Skipping merge for Other code: hgp_code_n_1225_k_25_r_1_Z_cz_code.json
Skipping merge for Other code: lp_code_n_2688_k_184_r_1_X_cz_code.json
Skipping merge for Other code: lp_code_n_1344_k_100_r_1_X_cz_code.json
Skipping merge for Other code: hgp_code_n_2401_k_49_r_1_Z_cz_code.json
Processing bb_code_n_180_k_8_r_1_Z_cz_code.json
Saving NA Stim circuit to /home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_na_stim_circuit/bb_code_n_180_k_8_r_1_Z_cz_na_without_measure.stim
Looking for original Stim file at /home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_stim_circuit/bb_code_n_180_k_8_r_1_Z.stim
Stim circuit saved to /home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_na_stim_circuit/bb_code_n_180_k_8_r_1_Z_cz_na.stim
Final Stim circuit saved as bb_code_n_180_k_8_r_1_Z_cz_na.stim
Processing bb_code_n_288_k_12_r_1_X_cz_code.json
Saving NA Stim circuit to /home/normaluser/ck/linkequantum-qLDPC/qLDPC/qldpc_na_stim_circuit/bb_code_n_288_k_12_r_1_X_cz_na_without_measure.stim
Looking for orig